In [1]:
# imports
import pandas as pd
import pickle as pkl
 

## Problem Statement

Using voting results and voter-registration and demographics from Orange County in 2014 general election, this study will create a classification model that will predict vote for Governor based on key demographic inputs.  

Dataset:  2 CSV files per year from https://statewidedatabase.org/d10/g14.html
1. Orange County (county 059) - SOV column - by srprec (voting results)
1. Orange County (county 059) - VOTE column - by srprec (voter reg & demographics)
1. For general elections in the years:  2012, 2014, 2016, 2018

To be merged on the ['srprec'] column.

In [5]:
sov = pd.read_csv('./vote_data/c059_g18_sov_data_by_g18_srprec.csv')

In [6]:
voters = pd.read_csv('./vote_data/c059_g18_voters_by_g18_srprec.csv')

In [7]:
sov.shape

(1547, 85)

In [8]:
voters.shape

(1334, 140)

### Combine datasets, aligned by ['srprec']

In [ ]:
sov['srprec']   

In [ ]:
# change sov['srprec'] to number
sov['srprec'] = pd.to_numeric(sov['srprec'], errors='coerce')

In [ ]:
sov['srprec']

In [ ]:
#find and drop nulls, non-conforming rows
sov['srprec'].isna().sum()

In [ ]:
sov['srprec'].dropna(inplace=True)

In [ ]:
sov['srprec']

In [ ]:
#change to integer to be congruent with voters df
sov['srprec'] = sov['srprec'].astype('Int64')

In [13]:
sov['srprec']

0       10316
1       10317
2       10319
3       10320
4       10325
        ...  
1542    75122
1543    75701
1544    75702
1545     7701
1546     <NA>
Name: srprec, Length: 1547, dtype: Int64

In [9]:
#get rid of non-conforming row.  
# Note:  May return errors.  
# IMPORTANT!:  Hand-coded for each year, based on results of cell above.
sov.drop(sov.index[1546], inplace=True)

In [15]:
voters['srprec']

0       10316
1       10317
2       10319
3       10320
4       10325
        ...  
1329    75117
1330    75122
1331    75701
1332    75702
1333     7701
Name: srprec, Length: 1334, dtype: int64

In [16]:
voters['srprec'].isna().sum()

0

### Examine Registration and Voter dataframes to align ['srprec'] and number of rows.

In [17]:
#combine two 'srprec' columns into a df to find uniques
sovprec_compare = sov['srprec']
sovprec_compare

0       10316
1       10317
2       10319
3       10320
4       10325
        ...  
1541    75117
1542    75122
1543    75701
1544    75702
1545     7701
Name: srprec, Length: 1546, dtype: Int64

In [18]:
voteprec_compare = voters['srprec']
voteprec_compare

0       10316
1       10317
2       10319
3       10320
4       10325
        ...  
1329    75117
1330    75122
1331    75701
1332    75702
1333     7701
Name: srprec, Length: 1334, dtype: int64

In [19]:
srprec_compare = pd.concat([sovprec_compare, voteprec_compare]).reset_index()

In [20]:
srprec_compare.shape

(2880, 2)

In [21]:
srprec_compare.head()

,index,srprec
0,0,10316
1,1,10317
2,2,10319
3,3,10320
4,4,10325


In [22]:
srprec_compare['srprec'].value_counts()

32270    2
2117     2
31005    2
2337     2
68073    2
        ..
65051    1
65065    1
42103    1
65061    1
70135    1
Name: srprec, Length: 1546, dtype: Int64

In [23]:
#merge the vote df (sov) with registration df (voters), keeping unique rows
result = pd.merge(left=sov, right=voters, how='outer', on=['srprec', 'srprec'], indicator=True)

In [24]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

result

,county,srprec,addist,cddist,sddist,...,oreg6g,oreg7g,oreg8g,oreg9g,_merge
0,30,10316,65,47,29,...,1.0,0.0,0.0,1.0,both
1,30,10317,65,47,29,...,2.0,2.0,4.0,1.0,both
2,30,10319,65,47,29,...,3.0,1.0,2.0,6.0,both
3,30,10320,65,47,29,...,4.0,1.0,3.0,9.0,both
4,30,10325,65,47,29,...,1.0,0.0,1.0,8.0,both
...,...,...,...,...,...,...,...,...,...,...,...
1541,30,75117,68,45,37,...,0.0,0.0,0.0,2.0,both
1542,30,75122,68,45,37,...,0.0,0.0,1.0,0.0,both
1543,30,75701,68,45,37,...,2.0,0.0,0.0,1.0,both
1544,30,75702,68,45,37,...,1.0,0.0,0.0,4.0,both


In [25]:
result['_merge'].value_counts()
# identifies 'left_only' rows from sov df that are unique

both          1334
left_only      212
right_only       0
Name: _merge, dtype: int64

In [26]:
#inspect the unique rows to understand if they hold any useful information
vote_only = result.loc[result['_merge'] == 'left_only']
pd.set_option('display.max_columns', None)
vote_only


,county,srprec,addist,cddist,sddist,bedist,TOTREG,DEMREG,REPREG,AIPREG,GRNREG,LIBREG,NLPREG,REFREG,DCLREG,MSCREG,TOTVOTE,DEMVOTE,REPVOTE,AIPVOTE,GRNVOTE,LIBVOTE,NLPVOTE,REFVOTE,DCLVOTE,MSCVOTE,PRCVOTE,ABSVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSLIB01,ASSREP01,ATGDEM01,ATGREP01,BOEDEM01,BOEREP01,CNGDEM01,CNGDEM02,CNGGRN01,CNGIND01,CNGREP01,CNGREP02,CONDEM01,CONREP01,GOVDEM01,GOVREP01,INSDEM01,INSIND01,LTGDEM01,LTGDEM02,PR_10_N,PR_10_Y,PR_11_N,PR_11_Y,PR_12_N,PR_12_Y,PR_1_N,PR_1_Y,PR_2_N,PR_2_Y,PR_3_N,PR_3_Y,PR_4_N,PR_4_Y,PR_5_N,PR_5_Y,PR_6_N,PR_6_Y,PR_7_N,PR_7_Y,PR_8_N,PR_8_Y,SENDEM01,SENDEM02,SENIND01,SENREP01,SOSDEM01,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01,USSDEM01,USSDEM02,election,type,totreg_r,dem,rep,aip,paf,msc,lib,nlp,grn,ref,dcl,male,female,hispdem,hisprep,hispdcl,hispoth,jewdem,jewrep,jewdcl,jewoth,kordem,korrep,kordcl,koroth,jpndem,jpnrep,jpndcl,jpnoth,chidem,chirep,chidcl,chioth,inddem,indrep,inddcl,indoth,vietdem,vietrep,vietdcl,vietoth,fildem,filrep,fildcl,filoth,demmunk,demm1824,demm2534,demm3544,demm4554,demm5564,demm65pl,demfunk,demf1824,demf2534,demf3544,demf4554,demf5564,demf65pl,repmunk,repm1824,repm2534,repm3544,repm4554,repm5564,repm65pl,repfunk,repf1824,repf2534,repf3544,repf4554,repf5564,repf65pl,dclmunk,dclm1824,dclm2534,dclm3544,dclm4554,dclm5564,dclm65pl,dclfunk,dclf1824,dclf2534,dclf3544,dclf4554,dclf5564,dclf65pl,othmunk,othm1824,othm2534,othm3544,othm4554,othm5564,othm65pl,othfunk,othf1824,othf2534,othf3544,othf4554,othf5564,othf65pl,dreg1g,dreg2g,dreg3g,dreg4g,dreg5g,dreg6g,dreg7g,dreg8g,dreg9g,rreg1g,rreg2g,rreg3g,rreg4g,rreg5g,rreg6g,rreg7g,rreg8g,rreg9g,ireg1g,ireg2g,ireg3g,ireg4g,ireg5g,ireg6g,ireg7g,ireg8g,ireg9g,oreg1g,oreg2g,oreg3g,oreg4g,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g,_merge
10,30,10342,65,47,29,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11,30,10343,65,47,29,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
57,30,13100,65,39,29,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
79,30,13380,6

In [27]:
#scan mean, min, max values to gauge data values

#expand display for quick visual assessment
pd.set_option('display.max_row', None)

vote_only.describe().T
#note:  returns ~200 rows of summary information

,count,mean,std,min,25%,50%,75%,max
county,212.0,30.000000,0.000000,30.0,30.00,30.0,30.00,30.0
srprec,212.0,48921.297170,19823.386787,2096.0,42101.75,53097.0,65052.25,72110.0
addist,212.0,69.452830,4.895364,55.0,68.00,68.0,73.00,74.0
cddist,212.0,45.669811,2.595007,39.0,45.00,45.0,48.00,49.0
sddist,212.0,35.566038,2.616096,29.0,36.00,37.0,37.00,37.0
bedist,212.0,4.000000,0.000000,4.0,4.00,4.0,4.00,4.0
TOTREG,212.0,0.023585,0.247088,0.0,0.00,0.0,0.00,3.0
DEMREG,212.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
REPREG,212.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0
AIPREG,212.0,0.000000,0.000000,0.0,0.00,0.0,0.00,0.0


In [33]:
#reset display constraints
pd.set_option('display.max_row', 15)

vote_only['TOTVOTE'].value_counts()
# only a few mis-entered data points of a handful of votes.  OK to drop the unique rows.

0    200
1      9
2      3
Name: TOTVOTE, dtype: int64

In [34]:
vote_only.loc[vote_only['TOTVOTE'] > 0]  

,county,srprec,addist,cddist,sddist,bedist,TOTREG,DEMREG,REPREG,AIPREG,GRNREG,LIBREG,NLPREG,REFREG,DCLREG,MSCREG,TOTVOTE,DEMVOTE,REPVOTE,AIPVOTE,GRNVOTE,LIBVOTE,NLPVOTE,REFVOTE,DCLVOTE,MSCVOTE,PRCVOTE,ABSVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSLIB01,ASSREP01,ATGDEM01,ATGREP01,BOEDEM01,BOEREP01,CNGDEM01,CNGDEM02,CNGGRN01,CNGIND01,CNGREP01,CNGREP02,CONDEM01,CONREP01,GOVDEM01,GOVREP01,INSDEM01,INSIND01,LTGDEM01,LTGDEM02,PR_10_N,PR_10_Y,PR_11_N,PR_11_Y,PR_12_N,PR_12_Y,PR_1_N,PR_1_Y,PR_2_N,PR_2_Y,PR_3_N,PR_3_Y,PR_4_N,PR_4_Y,PR_5_N,PR_5_Y,PR_6_N,PR_6_Y,PR_7_N,PR_7_Y,PR_8_N,PR_8_Y,SENDEM01,SENDEM02,SENIND01,SENREP01,SOSDEM01,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01,USSDEM01,USSDEM02,election,type,totreg_r,dem,rep,aip,paf,msc,lib,nlp,grn,ref,dcl,male,female,hispdem,hisprep,hispdcl,hispoth,jewdem,jewrep,jewdcl,jewoth,kordem,korrep,kordcl,koroth,jpndem,jpnrep,jpndcl,jpnoth,chidem,chirep,chidcl,chioth,inddem,indrep,inddcl,indoth,vietdem,vietrep,vietdcl,vietoth,fildem,filrep,fildcl,filoth,demmunk,demm1824,demm2534,demm3544,demm4554,demm5564,demm65pl,demfunk,demf1824,demf2534,demf3544,demf4554,demf5564,demf65pl,repmunk,repm1824,repm2534,repm3544,repm4554,repm5564,repm65pl,repfunk,repf1824,repf2534,repf3544,repf4554,repf5564,repf65pl,dclmunk,dclm1824,dclm2534,dclm3544,dclm4554,dclm5564,dclm65pl,dclfunk,dclf1824,dclf2534,dclf3544,dclf4554,dclf5564,dclf65pl,othmunk,othm1824,othm2534,othm3544,othm4554,othm5564,othm65pl,othfunk,othf1824,othf2534,othf3544,othf4554,othf5564,othf65pl,dreg1g,dreg2g,dreg3g,dreg4g,dreg5g,dreg6g,dreg7g,dreg8g,dreg9g,rreg1g,rreg2g,rreg3g,rreg4g,rreg5g,rreg6g,rreg7g,rreg8g,rreg9g,ireg1g,ireg2g,ireg3g,ireg4g,ireg5g,ireg6g,ireg7g,ireg8g,ireg9g,oreg1g,oreg2g,oreg3g,oreg4g,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g,_merge
220,30,2111,68,45,37,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
261,30,2303,65,46,29,4,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,0,2,0,2,0,0,0,0,0,2,0,2,0,2,0,1,1,0,2,1,1,0,2,1,1,0,2,0,2,0,2,2,0,0,2,0,2,0,2,0,0,0,0,2,0,0,2,2,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
605,30,39232,72,48,34,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
657,30,43120

In [35]:
result.shape


(1546, 225)

In [36]:
# drop 'left_only' rows
#df.drop(df.loc[df['line_race']==0].index, inplace=True)
# df[(df.line_race != 0) & (df.line_race != 10)]

result_both = result.drop(result.loc[result['_merge'] == 'left_only'].index)

In [37]:
result_both.shape

(1334, 225)

In [468]:
# save result-both df to preserve 

In [469]:
# result_both.to_pickle('./vote_data/merged_OC-c059_g18_by_srprec.pkl')
# result_both.to_pickle('./vote_data/merged_OC-c059_g16_by_srprec.pkl')
# result_both.to_pickle('./vote_data/merged_OC-c059_g14_by_srprec.pkl')
# result_both.to_pickle('./vote_data/merged_OC-c059_g12_by_srprec.pkl')

In [40]:
data = pd.read_pickle('./vote_data/merged_OC-c059_g18_by_srprec.pkl')
# data = pd.read_pickle('./vote_data/merged_OC-c059_g16_by_srprec.pkl')
# data = pd.read_pickle('./vote_data/merged_OC-c059_g14_by_srprec.pkl')
# data = pd.read_pickle('./vote_data/merged_OC-c059_g12_by_srprec.pkl')
data.shape

FileNotFoundError: [Errno 2] No such file or directory: './vote_data/merged_OC-c059_g18_by_srprec.pkl'

In [39]:
data.head()

,county,srprec,addist,cddist,sddist,bedist,TOTREG,DEMREG,REPREG,AIPREG,GRNREG,LIBREG,NLPREG,REFREG,DCLREG,MSCREG,TOTVOTE,DEMVOTE,REPVOTE,AIPVOTE,GRNVOTE,LIBVOTE,NLPVOTE,REFVOTE,DCLVOTE,MSCVOTE,PRCVOTE,ABSVOTE,ASSDEM01,ASSDEM02,ASSIND01,ASSLIB01,ASSREP01,ATGDEM01,ATGREP01,BOEDEM01,BOEREP01,CNGDEM01,CNGDEM02,CNGGRN01,CNGIND01,CNGREP01,CNGREP02,CONDEM01,CONREP01,GOVDEM01,GOVREP01,INSDEM01,INSIND01,LTGDEM01,LTGDEM02,PR_10_N,PR_10_Y,PR_11_N,PR_11_Y,PR_12_N,PR_12_Y,PR_1_N,PR_1_Y,PR_2_N,PR_2_Y,PR_3_N,PR_3_Y,PR_4_N,PR_4_Y,PR_5_N,PR_5_Y,PR_6_N,PR_6_Y,PR_7_N,PR_7_Y,PR_8_N,PR_8_Y,SENDEM01,SENDEM02,SENIND01,SENREP01,SOSDEM01,SOSREP01,SPINOP01,SPINOP02,TRSDEM01,TRSREP01,USSDEM01,USSDEM02,election,type,totreg_r,dem,rep,aip,paf,msc,lib,nlp,grn,ref,dcl,male,female,hispdem,hisprep,hispdcl,hispoth,jewdem,jewrep,jewdcl,jewoth,kordem,korrep,kordcl,koroth,jpndem,jpnrep,jpndcl,jpnoth,chidem,chirep,chidcl,chioth,inddem,indrep,inddcl,indoth,vietdem,vietrep,vietdcl,vietoth,fildem,filrep,fildcl,filoth,demmunk,demm1824,demm2534,demm3544,demm4554,demm5564,demm65pl,demfunk,demf1824,demf2534,demf3544,demf4554,demf5564,demf65pl,repmunk,repm1824,repm2534,repm3544,repm4554,repm5564,repm65pl,repfunk,repf1824,repf2534,repf3544,repf4554,repf5564,repf65pl,dclmunk,dclm1824,dclm2534,dclm3544,dclm4554,dclm5564,dclm65pl,dclfunk,dclf1824,dclf2534,dclf3544,dclf4554,dclf5564,dclf65pl,othmunk,othm1824,othm2534,othm3544,othm4554,othm5564,othm65pl,othfunk,othf1824,othf2534,othf3544,othf4554,othf5564,othf65pl,dreg1g,dreg2g,dreg3g,dreg4g,dreg5g,dreg6g,dreg7g,dreg8g,dreg9g,rreg1g,rreg2g,rreg3g,rreg4g,rreg5g,rreg6g,rreg7g,rreg8g,rreg9g,ireg1g,ireg2g,ireg3g,ireg4g,ireg5g,ireg6g,ireg7g,ireg8g,ireg9g,oreg1g,oreg2g,oreg3g,oreg4g,oreg5g,oreg6g,oreg7g,oreg8g,oreg9g,_merge
0,30,10316,65,47,29,4,1735,0,0,0,0,0,0,0,0,0,1278,0,0,0,0,0,0,0,0,0,0,0,610,0,0,0,606,602,618,524,650,630,0,0,0,592,0,624,592,590,665,475,684,466,548,841,373,378,821,502,706,693,509,556,657,674,511,539,661,566,619,537,681,454,741,799,414,0,0,0,0,625,591,595,478,605,611,441,630,g18,V,1238.0,423.0,517.0,24.0,1.0,0.0,6.0,0.0,2.0,0.0,265.0,582.0,656.0,55.0,43.0,30.0,6.0,17.0,19.0,14.0,0.0,18.0,26.0,24.0,1.0,12.0,17.0,11.0,0.0,12.0,15.0,20.0,2.0,9.0,0.0,4.0,0.0,0.0,4.0,2.0,0.0,8.0,4.0,9.0,0.0,0.0,9.0,19.0,23.0,29.0,26.0,52.0,0.0,17.0,35.0,36.0,43.0,50.0,84.0,0.0,18.0,15.0,26.0,36.0,73.0,89.0,0.0,5.0,16.0,28.0,38.0,70.0,103.0,0.0,14.0,20.0,27.0,36.0,25.0,26.0,0.0,10.0,15.0,15.0,22.0,24.0,31.0,0.0,0.0,4.0,6.0,4.0,4.0,1.0,0.0,2.0,2.0,1.0,4.0,4.0,1.0,105.0,78.0,11.0,26.0,13.0,25.0,13.0,31.0,121.0,96.0,69.0,12.0,28.0,24.0,26.0,20.0,33.0,209.0,96.0,53.0,14.0,17.0,7.0,13.0,7.0,16.0,42.0,16.0,8.0,2.0,2.0,3.0,1.0,0.0,0.0,1.0,both
1,30,10317,65,47,29,4,2079,0,0,0,0,0,0,0,0,0,1214,0,0,0,0,0,0,0,0,0,0,0,646,0,0,0,516,640,524,609,537,638,0,0,0,513,0,664,496,643,539,555,555,475,493,737,421,417,731,469,680,587,551,482,660,567,570,429,712,609,525,505,654,454,697,677,474,0,0,0,0,644,524,593,420,646,515,446,578,g18,V,1184.0,490.0,356.0,33.0,2.0,6.0,7.0,0.0,4.0,0.0,286.0,588.0,596.0,165.0,60.0,71.0,10.0,7.0,11.0,7.0,2.0,20.0,16.0,13.0,3.0,9.0,3.0,8.0,0.0,11.0,6.0,15.0,1.0,17.0,1.0,5.0,0.0,7.0,3.0,6.0,1.0,14.0,17.0,10.0,2.0,0.0,30.0,40.0,35.0,42.0,45.0,37.0,0.0,29.0,32.0,45.0,31.0,50.0,74.0,0.0,5.0,17.0,21.0,37.0,58.0,47.0,0.0,7.0,11.0,18.0,38.0,48.0,49.0,0.0,12.0,28.0,29.0,24.0,23.0,27.0,0.0,11.0,27.0,41.0,23.0,23.0,18.0,0.0,2.0,9.0,6.0,6.0,7.0,1.0,0.0,2.0,4.0,4.0,3.0,5.0,3.0,129.0,125.0,24.0,30.0,21.0,37.0,4.0,19.0,101.0,78.0,58.0,23.0,27.0,30.0,27.0,12.0,26.0,75.0,108.0,67.0,17.0,20.0,22.0,15.0,2.0,10.0,25.0,19.0,13.0,3.0,2.0,6.0,2.0,2.0,4.0,1.0,both
2,30,10319,65,47,29,4,1448,0,0,0,0,0,0,0,0,0,999,0,0,0,0,0,0,0,0,0,0,0,484,0,0,0,471,481,480,459,480,495,0,0,0,462,0,496,458,485,498,412,504,385,414,620,332,310,640,376,569,492,459,409,539,504,433,385,560,468,469,411,548,350,592,594,359,0,0,0,0,507,454,511,331,492,471,345,482,g18,V,945.0,316.0,335.0,22.0,2.0,3.0,7.0,0.0,2.0,2.0,256.0,443.0,502.0,68.0,37.0,49.0,5.0,12.0,8.0,5.0,0.0,19.0,6.0,

In [474]:
data['cddist'].value_counts()

45    347
48    334
39    226
46    198
47    133
49     87
38      9
Name: cddist, dtype: int64

In [473]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
